In [3]:
!pip install PyMuPDF spacy transformers scikit-learn fastapi uvicorn


  Using cached PyMuPDF-1.24.7-cp310-none-manylinux2014_x86_64.whl (3.5 MB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.4/62.4 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.7/15.7 MB 45.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.9/71.9 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.1/141.1 kB 11.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.7/307.7 kB 21.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 341.4/341.4 kB 31.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [4]:
import fitz  # PyMuPDF
import spacy
from transformers import pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Load Spacy model
nlp = spacy.load('en_core_web_sm')

# Define function to extract text from PDF
def extract_text_from_pdf(file_path):
    document = fitz.open(file_path)
    text = ""
    for page_num in range(len(document)):
        page = document.load_page(page_num)
        text += page.get_text()
    document.close()
    return text

# Define function to identify clauses
def identify_clauses(text):
    doc = nlp(text)
    clauses = [sent.text for sent in doc.sents]
    return clauses

# Define function for text classification
classifier = pipeline('text-classification', model='distilbert-base-uncased-finetuned-sst-2-english')

def classify_clauses(clauses):
    classified_clauses = []
    for clause in clauses:
        result = classifier(clause)
        classified_clauses.append((clause, result[0]['label']))
    return classified_clauses

# Define function for template comparison
def compare_with_template(clause, template_clauses):
    tfidf_vectorizer = TfidfVectorizer().fit_transform([clause] + template_clauses)
    cosine_matrix = cosine_similarity(tfidf_vectorizer[0:1], tfidf_vectorizer[1:])
    max_similarity_index = cosine_matrix.argmax()
    return template_clauses[max_similarity_index], cosine_matrix[0][max_similarity_index]

# Define function to highlight deviations
def highlight_deviations(clause, best_match):
    deviations = set(clause.split()) - set(best_match.split())
    highlighted_clause = clause
    for deviation in deviations:
        highlighted_clause = highlighted_clause.replace(deviation, f"**{deviation}**")
    return highlighted_clause


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

In [6]:
from google.colab import files

# Upload file
uploaded = files.upload()

# Assume single file upload
file_path = list(uploaded.keys())[0]

# Extract text from uploaded PDF
contract_text = extract_text_from_pdf(file_path)
print(contract_text)

# Identify clauses
clauses = identify_clauses(contract_text)
for i, clause in enumerate(clauses):
    print(f"Clause {i+1}: {clause}")


Saving SampleContract-Shuttle.pdf to SampleContract-Shuttle.pdf
 
Page 1 
Sample Contract 
 
Contract No.___________ 
PROFESSIONAL SERVICES AGREEMENT 
 
 
THIS AGREEMENT made and entered into this _______day of                      , 20     by and between the SANTA 
CRUZ COUNTY REGIONAL TRANSPORTATION COMMISSION, hereinafter called COMMISSION, and 
________    ____, hereinafter called CONSULTANT for __________________ (services/project name).   
 
1. DUTIES.  
A. CONSULTANT agrees to exercise special skill to accomplish the following results in a manner 
reasonably satisfactory to COMMISSION: ______________________________, as specified in Exhibit 
A: Scope of Services, which by this reference is incorporated herein. 
 
B. CONSULTANT shall provide the personnel listed below to perform the above-specified services, which 
persons are hereby designated as key personnel under this Agreement.   
 
Name  
 
 
Firm 
 
 
 
Function 
 
 
 
 
 
 
 
 
 
Principal in Charge 
 
 
 
 
 
 
 
 
 
Pro

In [5]:
!pip install PyPDF2
!pip install spacy
!python -m spacy download en_core_web_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 26.9 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [6]:
import PyPDF2
import spacy


def extract_text_from_pdf(pdf_path):
    with open(pdf_path, 'rb') as file:
        reader = PyPDF2.PdfReader(file)
        text = ''
        for page_num in range(len(reader.pages)):
            text += reader.pages[page_num].extract_text()
    return text


from google.colab import files
uploaded = files.upload()


pdf_path = next(iter(uploaded.keys()))
contract_text = extract_text_from_pdf(pdf_path)


import re

def preprocess_text(text):

    text = re.sub(r'\s+', ' ', text)
    return text

contract_text = preprocess_text(contract_text)


nlp = spacy.load('en_core_web_sm')

def extract_clauses_nlp(text):
    doc = nlp(text)
    clauses = []
    for sent in doc.sents:
        if 'Section' in sent.text or 'Article' in sent.text or 'Clause' in sent.text:
            clauses.append(sent.text.strip())
    return clauses

clauses_nlp = extract_clauses_nlp(contract_text)


for clause in clauses_nlp:
    print(clause)


Saving SampleContract-Shuttle.pdf to SampleContract-Shuttle (2).pdf
No person named in paragraph B of th is Section, or his or her successor, shall be removed or replaced by CONSULTANT, nor shall his or her agreed-upon func tion hereunder be changed, without the prior written consent of COMMISSION.
If CONSULTANT fails to submit the required Page 2 deliverable items according to the schedule set fort h in the Scope of Services, the COMMISSION may delay payment and/or terminate this Agreement in acc ordance with the provisions of Section 4 of this Agreement.
The CONSULTANT’s signature affixe d herein, and dated, shall constitute a certification under penalty of perjury under the laws of the St ate of California that the CONSULTANT has, unless exempt, complied with, the nondiscrimination program re quirements of Government Code Section 12990 and Title 2, California Administrative Code, Section 8103.
and Title 21, California C ode of Regulations, Chapter 21, Section 2500 et seq., if applic

In [7]:
!pip install PyPDF2
!pip install spacy
!pip install PyMuPDF
!python -m spacy download en_core_web_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 19.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.7/15.7 MB 27.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 49.2 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [8]:
import PyPDF2
import spacy
import fitz  # PyMuPDF

# Step 3: Load the PDF and extract text
def extract_text_from_pdf(pdf_path):
    with open(pdf_path, 'rb') as file:
        reader = PyPDF2.PdfReader(file)
        text = ''
        for page_num in range(len(reader.pages)):
            text += reader.pages[page_num].extract_text()
    return text

# Upload the PDF file to Colab
from google.colab import files
uploaded = files.upload()

# Assuming the uploaded file is named 'SampleContract-Shuttle.pdf'
pdf_path = next(iter(uploaded.keys()))
contract_text = extract_text_from_pdf(pdf_path)

# Step 4: Preprocess the text
import re

def preprocess_text(text):
    # Remove extra whitespace and newlines
    text = re.sub(r'\s+', ' ', text)
    return text

contract_text = preprocess_text(contract_text)

# Step 5: Use spaCy to extract clauses
nlp = spacy.load('en_core_web_sm')

def extract_clauses_nlp(text):
    doc = nlp(text)
    clauses = []
    for sent in doc.sents:
        if 'Section' in sent.text or 'Article' in sent.text or 'Clause' in sent.text:
            clauses.append(sent.text.strip())
    return clauses

clauses_nlp = extract_clauses_nlp(contract_text)

# Step 6: Highlight clauses in the PDF using PyMuPDF
def highlight_clauses_in_pdf(pdf_path, clauses):
    doc = fitz.open(pdf_path)
    for page_num in range(len(doc)):
        page = doc[page_num]
        for clause in clauses:
            text_instances = page.search_for(clause)
            for inst in text_instances:
                highlight = page.add_highlight_annot(inst)
                highlight.update()
    return doc

# Highlight clauses and save the modified PDF
highlighted_pdf = highlight_clauses_in_pdf(pdf_path, clauses_nlp)
highlighted_pdf.save("Highlighted_SampleContract-Shuttle.pdf")

# Download the modified PDF
files.download("Highlighted_SampleContract-Shuttle.pdf")

Saving SampleContract-Shuttle.pdf to SampleContract-Shuttle (3).pdf


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [10]:
import PyPDF2
import spacy
import fitz  # PyMuPDF

# Step 3: Load the PDF and extract text
def extract_text_from_pdf(pdf_path):
    with open(pdf_path, 'rb') as file:
        reader = PyPDF2.PdfReader(file)
        text = ''
        for page_num in range(len(reader.pages)):
            text += reader.pages[page_num].extract_text()
    return text

# Upload the PDF file to Colab
from google.colab import files
uploaded = files.upload()

# Assuming the uploaded file is named 'SampleContract-Shuttle.pdf'
pdf_path = next(iter(uploaded.keys()))
contract_text = extract_text_from_pdf(pdf_path)

# Step 4: Preprocess the text
import re

def preprocess_text(text):
    # Remove extra whitespace and newlines
    text = re.sub(r'\s+', ' ', text)
    return text

contract_text = preprocess_text(contract_text)

# Step 5: Use spaCy to extract clauses
nlp = spacy.load('en_core_web_sm')

def extract_clauses_nlp(text):
    doc = nlp(text)
    clauses = []
    for sent in doc.sents:
        if 'Section' in sent.text or 'Article' in sent.text or 'Clause' in sent.text:
            clauses.append(sent.text.strip())
    return clauses

clauses_nlp = extract_clauses_nlp(contract_text)

# Print template clauses to verify
for clause in clauses_nlp:
    print(clause)

# Step 6: Highlight clauses in the PDF using PyMuPDF
def highlight_text_in_pdf(pdf_path, clauses):
    doc = fitz.open(pdf_path)
    for page_num in range(len(doc)):
        page = doc[page_num]
        for clause in clauses:
            text_instances = page.search_for(clause)
            for inst in text_instances:
                highlight = page.add_highlight_annot(inst)
                highlight.update()
    return doc

# Highlight clauses and save the modified PDF
highlighted_pdf = highlight_text_in_pdf(pdf_path, clauses_nlp)
highlighted_pdf.save("HiiiSampleContract-Shuttle.pdf")

# Download the modified PDF
files.download("HiiiSampleContract-Shuttle.pdf")

Saving SampleContract-Shuttle.pdf to SampleContract-Shuttle (5).pdf
No person named in paragraph B of th is Section, or his or her successor, shall be removed or replaced by CONSULTANT, nor shall his or her agreed-upon func tion hereunder be changed, without the prior written consent of COMMISSION.
If CONSULTANT fails to submit the required Page 2 deliverable items according to the schedule set fort h in the Scope of Services, the COMMISSION may delay payment and/or terminate this Agreement in acc ordance with the provisions of Section 4 of this Agreement.
The CONSULTANT’s signature affixe d herein, and dated, shall constitute a certification under penalty of perjury under the laws of the St ate of California that the CONSULTANT has, unless exempt, complied with, the nondiscrimination program re quirements of Government Code Section 12990 and Title 2, California Administrative Code, Section 8103.
and Title 21, California C ode of Regulations, Chapter 21, Section 2500 et seq., if applic

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>